In [20]:
import numpy as np
import pandas as pd
from statsmodels.stats.weightstats import _zconfint_generic, _tconfint_generic
import scipy
from statsmodels.stats.proportion import proportion_confint, samplesize_confint_proportion
from matplotlib import pyplot as plt
from scipy import stats
import sklearn
from sklearn import model_selection
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from scipy.stats import sem, t
from statsmodels.stats.weightstats import *
from statsmodels.sandbox.stats.multicomp import multipletests 
import statsmodels.stats.multitest as smm
%matplotlib inline
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [7]:
df = pd.read_csv('gene_high_throughput_sequencing.csv')
df.head()

,Patient_id,Diagnosis,LOC643837,LOC100130417,SAMD11,NOC2L,KLHL17,PLEKHN1,C1orf170,HES4,...,CLIC2,RPS4Y1,ZFY,PRKY,USP9Y,DDX3Y,CD24,CYorf15B,KDM5D,EIF1AY
0,STT5425_Breast_001_normal,normal,1.257614,2.408148,13.368622,9.494779,20.880435,12.722017,9.494779,54.349694,...,4.761250,1.257614,1.257614,1.257614,1.257614,1.257614,23.268694,1.257614,1.257614,1.257614
1,STT5427_Breast_023_normal,normal,4.567931,16.602734,42.477752,25.562376,23.221137,11.622386,14.330573,72.445474,...,6.871902,1.815112,1.815112,1.815112,1.815112,1.815112,10.427023,1.815112,1.815112,1.815112
2,STT5430_Breast_002_normal,normal,2.077597,3.978294,12.863214,13.728915,14.543176,14.141907,6.232790,57.011005,...,7.096343,2.077597,2.077597,2.077597,2.077597,2.077597,22.344226,2.077597,2.077597,2.077597
3,STT5439_Breast_003_normal,normal,2.066576,8.520713,14.466035,7.823932,8.520713,2.066576,10.870009,53.292034,...,5.200770,2.066576,2.066576,2.066576,2.066576,2.066576,49.295538,2.066576,2.066576,2.066576
4,STT5441_Breast_004_normal,normal,2.613616,3.434965,12.682222,10.543189,26.688686,12.484822,1.364917,67.140393,...,11.227770,1.364917,1.364917,1.364917,1.364917,1.364917,23.627911,1.364917,1.364917,1.364917


In [3]:
df.shape

(72, 15750)

In [8]:
np.unique(df.Diagnosis)

array(['cancer', 'early neoplasia', 'normal'], dtype=object)

In [9]:
df_norm = df[df['Diagnosis'] == 'normal']
df_early_neoplasia = df[df['Diagnosis'] == 'early neoplasia']
df_cancer = df[df['Diagnosis'] == 'cancer']

In [ ]:
scipy.stats.ttest_ind()

In [11]:
gen_names = df.columns[2:]

In [18]:
norm_ear_canc = dict()
ear_canc_abs_cans = dict()
for name in gen_names:
    norm_ear_canc[name] = scipy.stats.ttest_ind(df_norm[name],df_early_neoplasia[name],equal_var=False)[1]
    ear_canc_abs_cans[name] = scipy.stats.ttest_ind(df_early_neoplasia[name],df_cancer[name],equal_var=False)[1]

In [19]:
counter_1 = 0
counter_2 = 0
for name in gen_names:
    if norm_ear_canc[name] < 0.05:
        counter_1+=1
    if ear_canc_abs_cans[name] < 0.05:
        counter_2+=1
print(counter_1,counter_2,sep=' ')

1575 3490


In [46]:
reject_1, p_corrected_1, a1_1, a2_1 = multipletests(list(norm_ear_canc.values()), 
                                            alpha = 0.025, 
                                            method = 'fdr_bh') 
reject_2, p_corrected_2, a1_2, a2_2 = multipletests(list(ear_canc_abs_cans.values()), 
                                            alpha = 0.025, 
                                            method = 'fdr_bh') 

In [38]:
control_norm = df_norm[gen_names].mean()
treatment_1 = df_early_neoplasia[gen_names].mean()
treatment_2 = df_cancer[gen_names].mean()

In [39]:
def Fold_ch(C,T):
    if T>C:
        return float(T)/C
    else:
        return float(-C)/T

In [40]:
norm_ear_canc_fc = dict()
ear_canc_abs_cans_fc = dict()
for name in gen_names:
    norm_ear_canc_fc[name] = Fold_ch(control_norm[name],treatment_1[name])
    ear_canc_abs_cans_fc[name] = Fold_ch(treatment_1[name],treatment_2[name])

In [42]:
st_conf_1 = list()
st_conf_2 = list()
for name in gen_names:
    if np.abs(norm_ear_canc_fc[name]) > 1.5:
        st_conf_1.append(name)
    if np.abs(ear_canc_abs_cans_fc[name]) > 1.5:
        st_conf_2.append(name)

In [47]:
p_corrected_1 = dict(zip(gen_names, p_corrected_1))
p_corrected_2 = dict(zip(gen_names, p_corrected_2))
p_corrected_1

{'LOC643837': 0.9665105247253503,
 'LOC100130417': 0.03569758201585074,
 'SAMD11': 0.5361027564569788,
 'NOC2L': 0.9807767931643708,
 'KLHL17': 0.49901595145307504,
 'PLEKHN1': 0.7029343906181045,
 'C1orf170': 0.8582825030907297,
 'HES4': 0.3865658877489498,
 'ISG15': 0.7943981337712143,
 'AGRN': 0.4530331252437427,
 'C1orf159': 0.36000761135258114,
 'LOC254099': 0.2622592898133774,
 'TTLL10': 0.9675008789456092,
 'TNFRSF18': 0.7382760987670549,
 'TNFRSF4': 0.8801323731358655,
 'SDF4': 0.720503092905841,
 'B3GALT6': 0.6377248039260699,
 'FAM132A': 0.220482646885068,
 'UBE2J2': 0.9730325778556353,
 'SCNN1D': 0.6839899552180817,
 'ACAP3': 0.7395845580077958,
 'PUSL1': 0.9470561452707192,
 'CPSF3L': 0.9775705573014093,
 'GLTPD1': 0.9949827201627851,
 'DVL1': 0.9588988974425776,
 'MXRA8': 0.9553360959512495,
 'AURKAIP1': 0.9014773519592366,
 'CCNL2': 0.9477485406866566,
 'LOC148413': 0.9702989488807887,
 'MRPL20': 0.8005396143299821,
 'LOC441869': 0.786845858614713,
 'VWA1': 0.262321664138

In [48]:
counter_1 = 0
counter_2 = 0
for name in st_conf_1:
    
    if p_corrected_1[name] < 0.025:
        counter_1 += 1
        
for name in st_conf_2:
    
    if p_corrected_2[name] < 0.025:
        counter_2 += 1
print(counter_1,counter_2,sep=' ')

4 524
